原文網址:
https://tlyu0419.github.io/2019/05/01/Crawl-Facebook/#more

In [1]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup

# 搜尋貼文連結
在這裡我們先定義一個函數，希望把網頁中各篇貼文的連結都找出來!<br>
ulr放我們要爬的Facebook網址，n是稍後要送出幾次滾動網頁到底部的命令，藉以加載更多資料。

### 碰到問題：
1. 瀑布式網頁：網頁不會一次把所有貼文都加載給你，當我們把頁面滾動到最下端時才會加載新貼文。
    * 解決方案：透過java指令讓視窗滾動到底部
2. 系統彈窗：當我們把頁面往下滾動後，會彈出一個請我們登入或註冊的視窗，阻礙我們爬資料的流程。
    * 解決方案：偵測「Not Now」的element位置，並透過程式點擊這個element

In [2]:
def FindLinks(url, n):
    Links = []
    driver.get(url)
    for i in range(n):
        time.sleep(2)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #滾動網頁到底部的命令
    driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click() # 這裡會跳出要求登入的畫面，找到「稍後再說」的按鈕並點擊
    soup = BeautifulSoup(driver.page_source)
    posts = soup.findAll('div', {'class':'clearfix y_c3pyo2ta3'})
    for i in posts:  #遍歷每個貼文的連結
        Links.append('https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]) 
    return Links

### 展開所有留言
定義一個展開所有留言的函數，透過while迴圈反覆搜尋與點擊「看更多留言」、「看更多回覆」與「看完整貼文內容」等按鈕。
在過程中會出現請我們登入或註冊的彈跳視窗，但我們不確定到底什麼時候會跳出，因此需要在過程中反覆偵測是有出現這個彈窗，若有就點擊「Not Now」

In [3]:
def expand(url):
    driver.get(url)
    try:
        driver.find_element_by_xpath('//a[@lang="en_US"]').click()
    except:
        print("Now is in EN_US")
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #過程中隨時可能會出現彈窗，若有就點擊「Not Now」
    
    # 點擊「comments」，藉以展開留言
    try:
        driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #過程中隨時可能會出現彈窗，若有就點擊「Not Now」
        time.sleep(1)
        driver.find_element_by_id('expanding_cta_close_button').click() 
    except:
        print('There is no comment!')
    k = 1
    while k != 0:
        k = 0
        for i in driver.find_elements_by_xpath('//div[@class="_5pcr userContentWrapper"]//div[@data-testid="UFI2CommentsList/root_depth_0"]//a[@role="button"]'): 
            # 反覆偵測是否有「看更多留言」、「看更多回覆」與「看完整貼文內容」等按鈕，若有擇點擊
            if bool(re.search('comment|More|Repl',i.text)) == True :
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #過程中隨時可能會出現彈窗，若有就點擊「Not Now」
                time.sleep(2)
                try:
                    driver.find_element_by_xpath('//div[@style="display: block;"]//a[@id="expanding_cta_close_button"]').click()
                except:
                    print('No pupup!')
                try:
                    i.click()
                except:
                    print('Nothing')
                time.sleep(2)
                k += 1

### 擷取貼文資訊與互動摘要
透過逐層搜索的方式，逐步定位我們要找的資訊
在這個環節需要反覆透過Chrome的功能比對資料，需要花一些心力進行比對
另外在這部分也使用了大量個try-except，原因是許多資料是有內容才會出現。例如並非每天貼人都會收到「哈哈」、「生氣」的心情。

In [5]:
# 文章內容與互動摘要
def PostContent(soup):
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # po文人資訊區塊
    PosterInfo = userContent.find('div', {'class':'l_c3pyo2v0u i_c3pynyi2f clearfix'})
    # 互動摘要區(讚、留言與分享)
    feedback = soup.find('form', {'class':'commentable_item collapsed_comments'})
    # 名稱
    Name = PosterInfo.find('img').attrs['aria-label']
    # ID
    ID = PosterInfo.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?',2)[0].split('/',-1)[-1]
    # 網址
    Link = driver.current_url
    # 發文時間
    try:
        Time = PosterInfo.find('abbr').attrs['title']
    except:
        Time = PosterInfo.find('div', {'class':'_1atc fsm fwn fcg'}).text
    # 文章內容
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = ""
    # Like
    try:
        Like = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label']
    except:
        Like = '0' 
    # Angry
    try:
        ANGER = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label']
    except:
        ANGER = '0'
    # HAHA
    try:
        HAHA = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label']
    except:
        HAHA = '0'
    # 留言
    try:
        commentcount = feedback.find('a', {'data-testid':'UFI2CommentsCount/root'}).text
    except:
        commentcount = '0' 
    # 分享
    try:
        share = feedback.find('span', {'class':'_355t _4vn2'}).text
    except:
        share = '0' 
    return pd.DataFrame(
        data = [{'Name':Name,
                 'ID':ID,
                 'Link':Link,
                 'Time':Time,
                 'Content':Content,
                 'Like':Like,
                 'ANGER':ANGER,
                 "HAHA":HAHA,
                 'commentcount':commentcount,
                 'share':share}],
        columns = ['Name', 'ID', 'Time', 'Content', 'Like', 'ANGER', 'HAHA', 'commentcount', 'share', 'Link'])

### 擷取粉絲留言資訊
這邊要留意雖然都是粉絲留言，但實際上分成「回應貼文的留言」與「回應留言的留言」。
函數中的第一個迴圈是用來抓「回應貼文的留言」，第二個則是抓「回應留言的留言」。讀者可以自行比較一下兩個迴圈中不同的地方。

In [6]:
# 留言
def CrawlComment(soup):
    Comments = pd.DataFrame()
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 用戶留言區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 回應貼文的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_0'}):
        try:
            CommentContent = i.find('span', {'dir':'ltr'}).text
        except:
            CommentContent = 'Sticker'
        Comment = pd.DataFrame(data = [{'CommentID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'CommentName':i.find('img').attrs['alt'],
                                 'CommentTime':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'CommentContent':CommentContent,
                                 'Link':driver.current_url}],
                        columns = ['CommentID', 'CommentName', 'CommentTime', 'CommentContent', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)
    
    # 回應留言的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
        try:
            CommentContent = i.find('span', {'dir':'ltr'}).text
        except:
            CommentContent = 'Sticker'
        Comment = pd.DataFrame(data = [{'CommentID':i.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'CommentName':i.find('img').attrs['alt'],
                                 'CommentTime':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'CommentContent':CommentContent,
                                 'Link':driver.current_url}],
                        columns = ['CommentID', 'CommentName', 'CommentTime', 'CommentContent', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)        
    return Comments

### 實作
今天要爬的是台灣大哥大的粉絲團頁面。暫時先設定加載20次資料，若想抓更多/更少資料的話可以自行調整n的數值。
注意這裡會透過Selenium開啟一個Chrome瀏覽器，若沒有下載的人可以參考在Windows上安裝Python & Selenium + 簡易教學。

In [8]:
driver = webdriver.Chrome()
Links = FindLinks(url = 'https://facebook.com/taiwanmobile/',
                  n = 20)
Links

['https://www.facebook.com/taiwanmobile/videos/774788163035648/',
 'https://www.facebook.com/taiwanmobile/posts/2680768658810225',
 'https://www.facebook.com/taiwanmobile/videos/1377136152458988/',
 'https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2680543115499446/',
 'https://www.facebook.com/taiwanmobile/posts/2676813149205776',
 'https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2676766872543737/',
 'https://www.facebook.com/taiwanmobile/posts/2676817049205386',
 'https://www.facebook.com/taiwanmobile/posts/2676701512550273',
 'https://www.facebook.com/taiwanmobile/posts/2676808395872918',
 'https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2676703935883364/',
 'https://www.facebook.com/taiwanmobile/posts/2676658972554527',
 'https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2676392685914489/',
 'https://www.facebook.com/taiwanmobile/posts/2664145983805826',
 'https://www.facebook.com/taiwanmobile/posts/2675122092708215',
 'h

In [ ]:
接著先創造兩個DataFrame，一個放文章內容，另一個放留言內容。
我讓程式自動輸出目前的處理的網址，若有無法抓出的頁面也會送出一個訊息提醒。方便我們後續追蹤哪裡出現錯誤。

In [9]:
# 抓下來所有留言
PostsInformation = pd.DataFrame()
PostsComments = pd.DataFrame()
for i in Links:
    print('Dealing with: ' + i)
    try:
        expand(i)
        soup = BeautifulSoup(driver.page_source)
        PostsInformation = pd.concat([PostsInformation, PostContent(soup)],ignore_index=True)
        PostsComments = pd.concat([PostsComments, CrawlComment(soup)],ignore_index=True)
    except:
        print('Load Failed: ' + i)

Dealing with: https://www.facebook.com/taiwanmobile/videos/774788163035648/
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2680768658810225
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/videos/1377136152458988/
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2680543115499446/
Now is in EN_US
Dealing with: https://www.facebook.com/taiwanmobile/posts/2676813149205776
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2676766872543737/
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2676817049205386
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2676701512550273
Now is in EN_US
Dealing with: https://www.facebook.com/taiwanmobile/posts/2676808395872918
Now is in EN_US
There is n

There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2650383305182094/
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2650408875179537
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2650271125193312
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2649170271970064
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2648997681987323
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2648363762050715
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2648199042067187/
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2646943168859441
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/ta

Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/posts/2620656784821413
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/videos/667886867369539/
Now is in EN_US
There is no comment!
Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2621741478046277/
Now is in EN_US
There is no comment!


In [ ]:
跑完之後我們就可以看到抓下來的結果囉！

In [10]:
PostsInformation

,Name,ID,Time,Content,Like,ANGER,HAHA,commentcount,share,Link
0,台灣大哥大與你在一起,taiwanmobile,2019/12/9 下午5:58,2019「台灣大哥大女子公開賽」賽前記者會直擊👉 網羅台巡賽前20名好手與外籍高手在球場拼高...,0,0,0,0,0,https://www.facebook.com/taiwanmobile/videos/7...
1,台灣大哥大與你在一起,taiwanmobile,2019/12/9 下午4:00,【🕗8點晨報】 隨著2020即將到來，預估各大手機品牌的上市商品也公開報告，身為手機寵兒的蘋...,38讚,0,0,1 則留言,4次分享,https://www.facebook.com/taiwanmobile/posts/26...
2,台灣大哥大與你在一起,taiwanmobile,2019/12/9 上午5:00,台灣大哥大匯流專案上線啦！ 上網這檔事啊～吃得巧還是不如吃得飽🍔🍔🍔 你生活所需的一切『光纖...,38讚,0,0,0,4次分享,https://www.facebook.com/taiwanmobile/videos/1...
3,台灣大哥大與你在一起,taiwanmobile,2019/12/9 上午12:10,#免費入場 #台巡賽一級賽事 #封關戰 這裡有喜歡打高爾夫的粉粉嗎？🙋‍♂️🙋‍♀️ 去年獲...,46讚,0,0,1 則留言,0,https://www.facebook.com/taiwanmobile/photos/a...
4,台灣大哥大與你在一起,taiwanmobile,2019/12/8 下午4:00,【🕗8點晨報】 天氣好冷，只想窩在家裡取暖，這時候追電影倒是個不錯的選擇，2019倒數計時，...,27讚,0,0,0,0,https://www.facebook.com/taiwanmobile/posts/26...
...,...,...,...,...,...,...,...,...,...,...
137,台灣大哥大與你在一起,taiwanmobile,2019/10/22 下午11:00,十月快過完了🇹🇼你MyCard了嗎 小額付費服務加碼優惠最後開催👉https://bit.l...,63讚,0,0,2則留言,3次分享,https://www.facebook.com/taiwanmobile/photos/a...
138,台灣大哥大與你在一起,taiwanmobile,2019/10/22 下午9:00,#君子動口不動手 你會不會常常兩手忙著吃東西沒空用手機？ 還是需要抱小孩空不出雙手？👶 或者...,65讚,0,2哈,1 則留言,4次分享,https://www.facebook.com/taiwanmobile/photos/a...
139,台灣大哥大與你在一起,taiwanmobile,2019/10/22 下午5:00,【🕗8點晨報】 如果還沒有更新到此版本的用戶可能要特別留意，舊的換機流程即將退場，如果不小心...,109讚,0,0,8則留言,21次分享,https://www.facebook.com/taiwanmobile/posts/26...
140,台灣大哥大與你在一起,taiwanmobile,2019/10/22 上午5:00,古人是揮一揮衣袖不帶走一片雲彩 我們是揮一揮手不留下一點指紋 #想不到吧徐志摩 不管是在跑步...,47讚,0,0,0,8次分享,https://www.facebook.com/taiwanmobile/videos/6...


In [11]:
PostsComments

,CommentID,CommentName,CommentTime,CommentContent,Link
0,100029495531249,梨子卓越,2019年12月10日 星期二上午10:36,Sticker,https://www.facebook.com/taiwanmobile/posts/26...
1,100002707922606,Alex Weng,2019年11月28日 星期四上午4:02,Google Nest Mini 台灣中文指令示範 說中文聽中文 (含中英雙語控制) En...,https://www.facebook.com/taiwanmobile/posts/26...
2,100002707922606,Alex Weng,2019年11月28日 星期四上午4:01,Google Nest Mini 台灣已發售 確定支援台灣中文指令 全台第一手實測請看這裡 ...,https://www.facebook.com/taiwanmobile/posts/26...
3,100003157239150,羅宇成,2019年12月6日 星期五上午8:40,讚,https://www.facebook.com/taiwanmobile/posts/26...
4,100010703624194,廖啟超,2019年12月6日 星期五下午1:55,台灣大，手機真的不夠多每次續都沒有想要的,https://www.facebook.com/taiwanmobile/posts/26...
...,...,...,...,...,...
133,100000615645466,林世陸,2019年10月23日 星期三上午11:09,來電不響怎摸辦,https://www.facebook.com/taiwanmobile/posts/26...
134,100010670130197,陳佳菁,2019年10月23日 星期三上午9:20,換機什麼意思,https://www.facebook.com/taiwanmobile/posts/26...
135,1448357445384692,台灣大哥大與你在一起,2019年10月23日 星期三下午3:08,這個要麻煩世陸詢問Line唷,https://www.facebook.com/taiwanmobile/posts/26...
136,100001319822483,劉家麟,2019年10月23日 星期三上午11:16,有支援esim 跟vowifi 嗎,https://www.facebook.com/taiwanmobile/photos/a...


In [ ]:
將資料保存到桌面，打開檔案的結果在文章的開頭，這裡就不再重複放囉!

In [12]:
PostsInformation.to_excel('FB_PostsInformation.xlsx')
PostsComments.to_excel('FB_PostsComments.xlsx')